In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(
    channel='ibm_quantum',
    instance='ibm-q/open/main',
    token='seu TOKEN no site da ibm'
)

# Or save your credentials on disk.
# QiskitRuntimeService.save_account(channel='ibm_quantum', instance='ibm-q/open/main', token='xxx')

In [ ]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.circuit import Parameter
from qiskit.visualization import plot_distribution
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import SamplerV2, EstimatorV2, QiskitRuntimeService

In [ ]:
QiskitRuntimeService().backends(simulator=False, operational=True)

In [ ]:
QiskitRuntimeService().least_busy(operational=True)

In [ ]:
backend_aer = AerSimulator()
backend_kyoto = QiskitRuntimeService().get_backend('ibm_kyoto')

In [ ]:
qc1 = QuantumCircuit(1)
qc1.h(0)
qc1.measure_all()
qc1.draw("mpl")

In [ ]:
th = Parameter('theta')
ph = Parameter('phi')

qr = QuantumRegister(2, name='q')
cr = ClassicalRegister(2, name='c')

qc2 = QuantumCircuit(qr,cr)
qc2.rx(th,1)
qc2.crx(ph,1,0)

qc2.measure(qr,cr)
qc2.draw("mpl")

In [ ]:
angles = [  [np.pi/4,np.pi/2] , [np.pi/3 , np.pi], [np.pi/2,np.pi]    ]

In [ ]:
qc2.parameters

In [ ]:
qc1_aer = transpile(qc1,backend=backend_aer)
qc1_aer.draw("mpl")

In [ ]:
qc2_aer = transpile(qc2,backend=backend_aer)
qc2_aer.draw("mpl")

In [ ]:
qc1_kyoto = transpile(qc1,backend=backend_kyoto)
qc1_kyoto.draw("mpl", idle_wires=False)

In [ ]:
qc2_kyoto = transpile(qc2,backend=backend_kyoto)
qc2_kyoto.draw("mpl", idle_wires=False)

In [ ]:
backend_aer = AerSimulator()
backend_kyoto = QiskitRuntimeService().get_backend('ibm_kyoto')

sampler_aer = SamplerV2(backend=backend_aer)
sampler_kyoto = SamplerV2(backend=backend_kyoto)

Sampler

In [ ]:
job_aer = sampler_aer.run( [ (qc1_aer,None,100),(qc2_aer,angles,1000)])

In [ ]:
job_kyoto = sampler_kyoto.run([(qc1_kyoto,None,100),(qc2_kyoto,angles,1000)]) #cstxtptsgar0008c01kg

In [ ]:
result_aer = job_aer.result()
result_kyoto = job_kyoto.result()

In [ ]:
counts_qc_aer = result_aer[0].data.meas.get_counts()
counts_qc_kyoto = result_kyoto[0].data.meas.get_counts()

In [ ]:
plot_distribution([counts_qc_aer,counts_qc_kyoto], legend=['Simulador', 'QPU'])

In [ ]:
counts_qc2_aer = result_aer[1].data.c.get_counts(0)
counts_qc2_kyoto = result_kyoto[1].data.c.get_counts(0)

#como rodamos um circuito parametrizado, ele mostrara o resultado da soma dos circuitos parametrizados
print(f'Aer: {counts_qc2_aer}')
print(f'QPU: {counts_qc2_kyoto}')

In [ ]:
#extraindo do primeiro circuito o resultados
counts_qc2_aer0 = result_aer[1].data.c.get_counts(1)
counts_qc2_kyoto0 = result_kyoto[1].data.c.get_counts(1)

plot_distribution([counts_qc2_aer0,counts_qc2_kyoto0], legend=['Simulador', 'QPU'])